# Flagging Loans Project


Assignment

The file `test_task.csv` contains data on repeated loans from current clients.

Using the available dataset, I will build a model that will predict the value of the target variable `bad_flag` and will argue for the choice of model chosen and show the expected default rate at different approval levels with detailed explanation.